**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [1]:
import corus
from corus import load_ne5

import re
import nltk

nltk.download('ieer')

import spacy
from spacy import displacy
import en_core_web_md

[nltk_data] Downloading package ieer to
[nltk_data]     C:\Users\gromo\AppData\Roaming\nltk_data...
[nltk_data]   Package ieer is already up-to-date!


In [2]:
dir = 'Collection5'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='001',
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[Ne5Span(
         index='T1',
         type='GEOPOLIT',
         start=0,
         st

In [3]:
document = next(records).text

nltk.pos_tag(nltk.word_tokenize(document))

[('Комиссар', 'JJ'),
 ('СЕ', 'NNP'),
 ('критикует', 'NNP'),
 ('ограничительную', 'NNP'),
 ('политику', 'NNP'),
 ('в', 'NNP'),
 ('отношении', 'NNP'),
 ('беженцев', 'NNP'),
 ('в', 'NNP'),
 ('европейских', 'NNP'),
 ('странах', 'VBD'),
 ('05/08/2008', 'CD'),
 ('10:32', 'CD'),
 ('МОСКВА', 'NN'),
 (',', ','),
 ('5', 'CD'),
 ('августа', 'JJ'),
 ('/Новости-Грузия/', 'JJ'),
 ('.', '.'),
 ('Проводимая', 'JJ'),
 ('в', 'JJ'),
 ('европейских', 'NN'),
 ('странах', 'NNP'),
 ('ограничительная', 'NNP'),
 ('политика', 'NNP'),
 ('в', 'NNP'),
 ('отношении', 'NNP'),
 ('беженцев', 'NNP'),
 ('нарушает', 'NNP'),
 ('ряд', 'NNP'),
 ('международных', 'NNP'),
 ('стандартов', 'NNP'),
 (',', ','),
 ('в', 'NNP'),
 ('частности', 'NNP'),
 (',', ','),
 ('право', 'NNP'),
 ('на', 'NNP'),
 ('воссоединение', 'NNP'),
 ('семей', 'NNP'),
 (',', ','),
 ('заявляет', 'NNP'),
 ('Комиссар', 'NNP'),
 ('Совета', 'NNP'),
 ('Европы', 'NNP'),
 ('по', 'NNP'),
 ('правам', 'NNP'),
 ('человека', 'NNP'),
 ('Томас', 'NNP'),
 ('Хаммарберг', '

In [4]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Thomas Hammarberg', 'PERSON'),
 ('Комиссар', 'PERSON'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('СЕ', 'ORGANIZATION'),
 ('Совета Европы', 'PERSON'),
 ('Хаммарберг', 'PERSON')}

**Spacy**

In [5]:
nlp = en_core_web_md.load()
doc = nlp(document)

displacy.render(doc, style="dep") # (1)
displacy.render(doc, style="ent") # (2)

In [6]:
from razdel import tokenize

In [7]:

words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [8]:
import pandas as pd

In [9]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [10]:
df_words['tag'].value_counts()

None        218933
PER          21189
ORG          13644
LOC           4567
GEOPOLIT      4340
MEDIA         2477
Name: tag, dtype: int64

In [11]:
df_words.head(3)

,word,tag
0,Пулеметы,None
1,",",None
2,автоматы,None


In [12]:
df_words.shape

(265150, 2)

In [13]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, Conv2D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [14]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [15]:
train_x.apply(len).max(axis=0)

55

In [16]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [18]:
def custom_standardization(input_data):
    return input_data

def data_preparation(train_data, seq_len=1, vocab_size = 30000):    

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [19]:
embedding_dim = 64

In [20]:
class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [21]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 1, vocab_size=vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12429/12429 [==============================] - 183s 15ms/step - loss: 0.2982 - accuracy: 0.9124 - val_loss: 0.2195 - val_accuracy: 0.9335
Epoch 2/5
12429/12429 [==============================] - 183s 15ms/step - loss: 0.1397 - accuracy: 0.9564 - val_loss: 0.2173 - val_accuracy: 0.9354
Epoch 3/5
12429/12429 [==============================] - 182s 15ms/step - loss: 0.1263 - accuracy: 0.9584 - val_loss: 0.2226 - val_accuracy: 0.9351
Epoch 4/5
12429/12429 [==============================] - 182s 15ms/step - loss: 0.1218 - accuracy: 0.9591 - val_loss: 0.2434 - val_accuracy: 0.9346
Epoch 5/5
12429/12429 [==============================] - 178s 14ms/step - loss: 0.1191 - accuracy: 0.9596 - val_loss: 0.2820 - val_accuracy: 0.8836


In [22]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 5, vocab_size=vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12429/12429 [==============================] - 178s 14ms/step - loss: 0.3055 - accuracy: 0.9101 - val_loss: 0.2220 - val_accuracy: 0.9329
Epoch 2/5
12429/12429 [==============================] - 179s 14ms/step - loss: 0.1423 - accuracy: 0.9555 - val_loss: 0.2307 - val_accuracy: 0.9355
Epoch 3/5
12429/12429 [==============================] - 179s 14ms/step - loss: 0.1274 - accuracy: 0.9583 - val_loss: 0.2269 - val_accuracy: 0.9354
Epoch 4/5
12429/12429 [==============================] - 175s 14ms/step - loss: 0.1225 - accuracy: 0.9592 - val_loss: 0.2286 - val_accuracy: 0.9350
Epoch 5/5
12429/12429 [==============================] - 175s 14ms/step - loss: 0.1199 - accuracy: 0.9596 - val_loss: 0.2409 - val_accuracy: 0.9333


In [23]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 10, vocab_size=vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12429/12429 [==============================] - 180s 14ms/step - loss: 0.3095 - accuracy: 0.9079 - val_loss: 0.2224 - val_accuracy: 0.9332
Epoch 2/5
12429/12429 [==============================] - 177s 14ms/step - loss: 0.1437 - accuracy: 0.9553 - val_loss: 0.2985 - val_accuracy: 0.8855
Epoch 3/5
12429/12429 [==============================] - 173s 14ms/step - loss: 0.1275 - accuracy: 0.9585 - val_loss: 0.2513 - val_accuracy: 0.9349
Epoch 4/5
12429/12429 [==============================] - 175s 14ms/step - loss: 0.1226 - accuracy: 0.9591 - val_loss: 0.2507 - val_accuracy: 0.9347
Epoch 5/5
12429/12429 [==============================] - 179s 14ms/step - loss: 0.1201 - accuracy: 0.9595 - val_loss: 0.2566 - val_accuracy: 0.9337


Длина подаваемой последовательности практически не влияет на результат.